# Incremental Dynamic Learning

## Overview

The idea behind Incremental Dynamic Learning networks is that information is not always learned at once, up front. As information is encountered, we train on the new data. Adding the new information to our existing knowlege base. Humans, are said to be able to recognize an image in 13 milliseconds after seeing it.The new image is added to their existing learned information and they do not have to retrain on all images they have ever seen.

## Example

As an example of how this can be used in the real world, I will use field biologists for my use case.

Imagine a group of field biologists are in the field and all have a mobile application which allows them to photograph wildlife they encounter, label it and forward it to a central system. The central system takes in the new photographs and trains a neural network on each piece of new data that comes in. W/out having to retrain a new model based on all past photographs. 

## Experiment 1

The experiment will consist of using a pre-trained network from TensorFlow (VGG) to generate our classification data. Our data will consist of using flower images provided with TensorFlow examples.

Once the data has be converted by the VGG network, the process of training our network begins. This will be done by feeding the network 1 image at a time, allowing the network to train for a short duration (13ms) and then move onto the next image. A fixed validation set will be used to monitor progress as the images are fed one at a time to the network.

The goal is to get an acceptable accuracy, similar to what a traditionally trained network would achieve.

### General Imports

## Network Construction

The firs thing we will do is construct our networks. We will use 3 different network models for this experiment:
* The Convolutional Network (VGG16) - Used to convolve our images
* The Classification Network (Dence Network) - Used to validate our training
* Full model network (Convolutional + Classification) - Used to train our Classification network.

The full model will actually be made of of layers from the Convolutional network and the Classification network. It will perform the job of convolving the images and classifying them. This is the model we will use to train our network.

These models are separated to allow the experiment to be run more efficiently and will be used for different functions thruout the experiment. By convolving our validation and test data up front, we will eliminate a good deal of redundent processing of the same data. This will allow us not to perform a validation step during each training epoch, and give us the flexibility of performing a validation step after some number of images are trained. Thus allowing us to monitor the progress of our work.

One thing to note is that we will make all the VGG16 layers not trainable and we will not include the top layer (because we are providing our own classifier.) This will allow us to feed our images into the full model, but train only the classification layer.

In [ ]:
import os
from os.path import isfile, isdir
from tqdm import tqdm
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from tensorflow.contrib.keras import preprocessing

### Get sample data

Let's pull down some sample data using photos of flowers provided by TensorFlow.

In [ ]:
import tarfile

dataset_folder_path = 'flower_photos'

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile('flower_photos.tar.gz'):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='Flowers Dataset') as pbar:
        urlretrieve(
            'http://download.tensorflow.org/example_images/flower_photos.tgz',
            'flower_photos.tar.gz',
            pbar.hook)

if not isdir(dataset_folder_path):
    with tarfile.open('flower_photos.tar.gz') as tar:
        tar.extractall()
        tar.close()

### Read a sample image to test our networks

In [ ]:
data_dir = 'flower_photos/'

test_img = preprocessing.image.load_img(data_dir + 'sunflowers/6953297_8576bf4ea3.jpg', target_size=(224,224,3))
test_img = preprocessing.image.img_to_array(test_img)
test_img = test_img.astype(np.uint8) 

print ('Image shape: {}'.format(test_img.shape))
plt.imshow(test_img)

### Deep Learning Imports

In [ ]:
from tensorflow.contrib.keras import models
from tensorflow.contrib.keras import layers
from tensorflow.contrib.keras import optimizers
from tensorflow.contrib.keras import applications

### Convolutional Network

In [ ]:
def build_conv_model(input_shape=(224,224,3), summary=False):

    model = applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=input_shape, pooling='avg')

    for layer in model.layers:
        layer.trainable = False

    if summary:
        model.summary()
    return model

conv_model = build_conv_model(summary=True)

### Use the conv model to convolve the image

In [ ]:
conv_model = build_conv_model()
feed_image = np.expand_dims(test_img, axis=0)
convolved_img = conv_model.predict(feed_image)

print ('Convolved image shape: {}'.format(convolved_img.shape))

conv_out_layer = conv_model.layers[-1]

print ('Shape of output layer is {}'.format(conv_out_layer.get_output_at(0).get_shape()))

### Classification Network

Here we are free to experiment with various dense layers and configurations. I have found initializing both the weights and the biases to zero produces good resluts. Also a low learning rate worked very well in producing more accurate results.

In [ ]:
def build_classifcation_model(conv_model, learning_rate=0.0001, classes=5, summary=False):
    model = models.Sequential()
    
    input_shape = conv_model.layers[-1].output_shape[1]
    model.add(layers.Dense(256, input_shape=[input_shape]))
    model.add(layers.Dense(classes, 
                           kernel_initializer='zeros', 
                           bias_initializer='zeros'))

    model.add(layers.Activation('softmax', name='out'))
    optimizer = optimizers.Adam(lr=learning_rate)

    model.compile(optimizer=optimizer,
                  loss='mean_squared_error',
                  metrics=['accuracy'])
    
    if summary:
        model.summary()
    return model

conv_model = build_conv_model()
classification_model = build_classifcation_model(conv_model, summary=True)

### Use the classification model to train on an convolved image

In [ ]:
conv_model = build_conv_model()
classification_model = build_classifcation_model(conv_model)
feed_image = np.expand_dims(test_img, axis=0)
convolved_img = conv_model.predict(feed_image)
label = [0, 0, 0, 1, 0]
label = np.expand_dims(label, axis=0)
classification_model.fit(convolved_img, label, epochs=1)

pred = classification_model.predict_classes(convolved_img)
print ('prediction is {}'.format(pred))

### Construct full network using layers from conv and classifcation models

In [ ]:
def build_full_model(conv_model, classification_model, learning_rate=0.0001, summary=False):
    # Allocate pre-built model and make non-trainable
    model = models.Sequential()
    for layer in conv_model.layers:
        model.add(layer)
    
    for layer in classification_model.layers:
        model.add(layer)
    
    optimizer = optimizers.Adam(lr=learning_rate)

    model.compile(optimizer=optimizer,
                  loss='mean_squared_error',
                  metrics=['accuracy'])
    
    if summary:
        model.summary()
    return model

conv_model = build_conv_model()
classification_model = build_classifcation_model(conv_model)
full_model = build_full_model(conv_model, classification_model, summary=True)

### Train the full model on the original image

In [ ]:
conv_model = build_conv_model()
classification_model = build_classifcation_model(conv_model)
full_model = build_full_model(conv_model, classification_model)
feed_image = np.expand_dims(test_img, axis=0)
label = [0, 0, 0, 1, 0]
label = np.expand_dims(label, axis=0)
full_model.fit(feed_image, label, epochs=1)

pred = full_model.predict_classes(feed_image)
print ('full prediction is {}'.format(pred))

# Demonstrate the classification model was also trained
convolved_img = conv_model.predict(feed_image)
pred = classification_model.predict_classes(convolved_img)
print ('classification prediction is {}'.format(pred))

## Read all the images, reshape the images and create labels for them

In [ ]:
from tensorflow.contrib.keras import preprocessing

data_dir = 'flower_photos/'
contents = os.listdir(data_dir)
classes = [each for each in contents if os.path.isdir(data_dir + each)]

# Set the batch size higher if you can fit in in your GPU memory
images = []
labels = []
batch = []

for each in classes:
    print("Starting {} images".format(each))
    class_path = data_dir + each
    files = os.listdir(class_path)
    for ii, file in enumerate(files, 1):
        # Add images to the current batch
        # utils.load_image crops the input images for us, from the center
        img = preprocessing.image.load_img(os.path.join(class_path, file), target_size=(224,224,3))
        img = preprocessing.image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = img.astype(np.uint8) 
        images.append(img)
        labels.append(each)

        # Display progress
        if ii % 250 == 0 or ii == len(files):
            print('{} images processed'.format(ii))
            
images = np.concatenate(images)

print ('Image loading complete: {}'.format(images.shape))

In [ ]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
lb.fit(labels)

labels_vecs = lb.transform(labels)
print (labels_vecs.shape)

## Generate the training, validation and test data sets

In [ ]:
from sklearn.model_selection import  train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, labels_vecs, train_size=0.8, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
sample_image = X_train[1000]
sample_label = y_train[1000]
print (sample_label)
print (classes[np.argmax(sample_label)])
plt.imshow(sample_image)

In [ ]:
datagen = preprocessing.image.ImageDataGenerator(
        rotation_range=5,
        width_shift_range=0.01,
        height_shift_range=0.01,
        shear_range = 0.01,
        zoom_range=0.2,
        horizontal_flip=False,
        fill_mode='nearest')

def gen_batch(image, label, count=9):
    batch_data = []
    batch_lables = []
    batch_data.append(image)
    batch_lables.append(label.tolist())
    for X_batch in datagen.flow(x=image):
        gen_image = X_batch[0]
        gen_image = gen_image.reshape((1,) + gen_image.shape)
        gen_image = gen_image.astype(np.uint8) 
        batch_data.append(gen_image)
        batch_lables.append(label.tolist())
        if len(batch_data) > count - 1:
            return batch_data, batch_lables
        
batch_images, batch_labels = gen_batch(first_image, first_label, count=2)

In [ ]:
print(first_label)
plt.imshow(first_image[0])
plt.show()


batch_images, batch_labels = gen_batch(first_image, first_label, count=9)
i = 0
print (batch_labels)
for image, label in zip(batch_images, batch_labels):
    plt.subplot(330 + 1 + i)
    print (label)
    plt.imshow(image[0])
    i += 1
    
# show the plot    
plt.show()

In [ ]:
from tensorflow.contrib.keras import applications

model = applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=None)

In [ ]:
model.summary()

In [ ]:
from tensorflow.contrib.keras import models
from tensorflow.contrib.keras import layers
from tensorflow.contrib.keras import optimizers
from tensorflow.contrib.keras import applications

def build_flower_classifcation_model(learning_rate=0.001, classes=5):
    # Allocate pre-built model and make non-trainable
    model = models.Sequential()
    vgg = applications.vgg16.VGG16(include_top=False, weights='imagenet', classes=classes, input_shape=(224, 224, 3))
    for layer in vgg.layers:
        trainable = False
        layer.trainable = trainable
        model.add(layer)
    
    model.add(layers.Flatten())
    model.add(layers.Dense(256))
    model.add(layers.Dropout(.5))
    model.add(layers.Dense(classes, kernel_initializer='zeros', bias_initializer='zeros'))

    model.add(layers.Activation('softmax', name='out'))
    optimizer = optimizers.Adam(lr=learning_rate)

    model.compile(optimizer=optimizer,
                  loss='mean_squared_error',
                  metrics=['accuracy'])
    model.summary()
    return model

model = build_flower_classifcation_model(learning_rate=0.0001, classes=len(labels_vecs[0]))

In [ ]:
for index in range(len(X_train)):
    single_image = X_train[index]
    single_label = y_train[index]
    batch_images, batch_labels = gen_batch(single_image, single_label, count=9)
    print (batch_labels)
    if index % 1 == 0:
        verbose = True
    else:
        verbose = False
    
    print ('Image: ', index)
    batch_images = np.concatenate(batch_images)
    batch_val_images = np.concatenate(X_val)
    model.fit(batch_images, batch_labels, validation_data=(batch_val_images, y_val), batch_size=len(batch_images), epochs=1, verbose=verbose)

batch_text = np.concatenate(X_test)
print (model.evaluate(batch_text, y_test))

In [ ]:
this_train = np.concatenate(X_train)
print (this_train.shape)
print (y_train.shape)
#model.fit(this_train, y_train, validation_data=(X_val, y_val), batch_size=8, epochs=1, verbose=verbose)
model.fit(this_train, y_train, batch_size=8, epochs=1, verbose=verbose)